#### Описание данных:<br> 
assessments.csv — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).<br>
code_module — идентификационный код предмета.<br>
code_presentation — семестр (Идентификационный код).<br>
id_assessment — тест (Идентификационный номер ассессмента).<br>
assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).<br>
date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).<br>
weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.<br>
<br>
courses.csv — файл содержит список предметов по семестрам.<br>
code_module — предмет (идентификационный код).<br>
code_presentation — семестр (идентификационный код).<br>
module_presentation_length — продолжительность семестра в днях.<br>
<br>
studentAssessment.csv — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.<br>
id_assessment — тест (идентификационный номер).<br>
id_student — идентификационный номер студента.<br>
date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.<br>
is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).<br>
score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.<br>
<br>
studentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.<br>
code_module — предмет (идентификационный код).<br>
code_presentation — семестр (идентификационный код)<br>
id_student — идентификационный номер студента.<br>
date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).<br>
date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from urllib.parse import urlencode

In [2]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
assessments_public_key = 'https://disk.yandex.ru/d/iSVP1sZ7AdW7TQ'
courses_public_key = 'https://disk.yandex.ru/d/jmJxR8yhwCXlaw'
studentAssessment_public_key = 'https://disk.yandex.ru/d/DYd4M5b_8RETsA'
studentRegistration_public_key = 'https://disk.yandex.ru/d/lqHnH8NOZ458iw'

In [3]:
def download_csv(public_key):
    
    final_url = base_url + urlencode(dict(public_key=public_key)) 
    response = requests.get(final_url) 
    download_url = response.json()['href'] 
    return download_url

In [4]:
df_assessments = pd.read_csv(download_csv(assessments_public_key))
df_courses = pd.read_csv(download_csv(courses_public_key))
df_studentAssessment = pd.read_csv(download_csv(studentAssessment_public_key))
df_studentRegistration = pd.read_csv(download_csv(studentRegistration_public_key))

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) 

курс - это один предмет за один семестр, то есть каждый экзамен - за пройденный курс

In [5]:
df_exam = df_assessments.query('assessment_type == "Exam"') #отбираю данные с с типом теста - экзамен

In [6]:
df_score_for_exam = df_studentAssessment \
                    .merge(df_exam, on='id_assessment') \
                    .drop(columns = ['is_banked', 'weight']) 
#объединяю таблицу с указанным типом теста и таблицу с данными об оценке учащегося в этом тесте

In [38]:
df_passed_one_exam = df_score_for_exam.query('score >= 40')\
                    .groupby('id_student', as_index=False)\
                    .agg({'id_assessment':'nunique'})\
                    .query('id_assessment == 1')
df_passed_one_exam.id_student.count()
#отбираю данные, удвлетворяющие условию (успешная сдача экзамена, т.е. оценка выше 40 + сдан только один экзамен)

3802

ОТВЕТ: 3802 студенда успешно сдали только один курс

2. Необходимо выявить самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью.<br>   
завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [10]:
df_passed_exam = df_score_for_exam.query('score >= 40')

In [78]:
successful_exams = df_passed_exam.groupby(['code_module', 'id_assessment'])\
                                    .agg({'score' : 'count'})
# Определение количества успешных попыток сдачи тестов

In [77]:
all_exams = df_score_for_exam.groupby(['code_module', 'id_assessment'])\
                            .agg({'score' : 'count'}) 
# Определение общего кол-ва попыток сдачи тестов

In [79]:
difference = successful_exams/all_exams
difference = difference.rename(columns={"score": "completion_rate"})
difference.sort_values('completion_rate')
# Расчет уровня завершаемости

completion_rate
code_module id_assessment                 
DDD         25340                 0.837209
CCC         24299                 0.872432
DDD         25368                 0.886316
CCC         24290                 0.888889
DDD         25354                 0.907025
            25361                 0.925573

In [80]:
easiest_exam = difference['completion_rate'].idxmax()
most_difficult_exam = difference['completion_rate'].idxmin()

In [83]:
print('Код предмета и ID теста с самой низкой завершаемостью:', *easiest_exam, '\nКод предмета и ID теста с самой высокой завершаемостью:', *most_difficult_exam)

Код предмета и ID теста с самой низкой завершаемостью: DDD 25361 
Код предмета и ID теста с самой высокой завершаемостью: DDD 25340


ОТВЕТ: самый простой экзамен - assessment 25361 по курсу предмета DDD в модуле 2014B,<br>
самый сложный - assessment 25340 по курсу предмета DDD в модуле 2013B

3. По каждому предмету определить средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом).

In [18]:
df_exam_avg = df_score_for_exam.query('score >= 40')\
                .groupby('code_module')\
                .agg({'date_submitted':'mean'})\
                .round(2)
df_exam_avg = df_exam_avg.rename(columns={"date_submitted": "avg_time_per_exam"})
df_exam_avg

,avg_time_per_exam
code_module,
CCC,239.35
DDD,237.98


ОТВЕТ: средний срок сдачи экзаменов по предмету CCC - 239.35, DDD - 237.98

4. Выявить самые популярные предметы (ТОП-3) по количеству регистраций на них.<br> 
А также предметы с самым большим оттоком (ТОП-3).

In [21]:
df_Registration = df_studentRegistration.groupby('code_module', as_index=False)\
                .agg({'date_registration':'count'})\
                .sort_values('date_registration')

In [22]:
df_unRegistration = df_studentRegistration.dropna(subset =['date_unregistration'])
df_unRegistration.isna().sum()

# существует 39 записей в таблице, согласно которым отмена записи на курс происошла без зафиксированной даты записи. 
# Можно считать, что произошла ошибка в записи данных и дата записи на курс в этих случаях не была зафиксирована. 

code_module             0
code_presentation       0
id_student              0
date_registration      39
date_unregistration     0
dtype: int64

In [23]:
df_Unregistration = df_unRegistration.groupby('code_module', as_index=False)\
                .agg({'date_unregistration':'count'})\
                .sort_values('date_unregistration')

ОТВЕТ: ТОП-3 самых популярных предмета: BBB, FFF, DDD.<br> 
ТОП-3 предмета с самым большим оттоком: FFF, BBB, DDD.

5. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. 

In [26]:
df_all_attempts = df_score_for_exam.groupby('code_presentation')\
                .agg({'score':'count'})

In [27]:
df_passed_exams = df_score_for_exam.query('score >= 40')\
                .groupby('code_presentation')\
                .agg({'score':'count'})

In [28]:
df_difference = (df_passed_exams / df_all_attempts).sort_values('score')

In [29]:
df_avg = df_score_for_exam.query('score >= 40')\
        .groupby('code_presentation')\
        .agg({'date_submitted':'mean'})\
        .sort_values('date_submitted')

ОТВЕТ: семестр с самой низкой завершаемостью курсов - 2013B<br>
семестр с самыми долгими средними сроками сдачи курсов - 2014J

6.RFM-анализ

R - среднее время сдачи студентом одного экзамена,<br> 
F - завершаемость курсов (* завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен),<br>
M - среднее количество баллов, получаемое за экзамен<br>

In [33]:
# (R) рассчитываю среднее время сдачи студентом одного экзамена от начала семестра и до последней сдачи (первым предположением было рассчитать время от первой попытки сдачи экзамена до последней успешной попытки, но случаев пересдачи выявлено не было)
df_exam_avg_student = df_score_for_exam.groupby(['id_student'], as_index=False)\
                 .agg({'date_submitted': 'mean'})

In [102]:
exam_attempts = df_score_for_exam.groupby(['id_student'])\
               .agg({'score':'count'})
exam_attempts.head()
# нахожу количество всех попыток сдать экзамен для каждого студента

,score
id_student,
23698,1
24213,1
27116,1
28046,1
28787,1


In [101]:
successful_attempts = df_score_for_exam.query('score >= 40')\
               .groupby(['id_student'])\
               .agg({'score':'count'})
successful_attempts.head()
# нахожу количество удачных попыток сдать экзамен для каждого студента

,score
id_student,
23698,1
24213,1
27116,1
28046,1
28787,1


In [103]:
# (F) вычисляю завершаемость курсов для каждого студента
completion_rate = successful_attempts / exam_attempts
completion_rate = completion_rate.fillna(0).rename(columns={'score':'completion_rate'})

In [104]:
# (M) нахожу среднюю оценку за экзамен для каждого студента (все оценки за всее экзамены, включая неудачные)
df_score_avg_student = df_score_for_exam.groupby('id_student', as_index=False)\
                        .agg({'score':'mean'})

In [105]:
df_rfm = df_exam_avg_student.merge(completion_rate, on='id_student')\
         .merge(df_score_avg_student, on='id_student')\
         .rename(columns={'date_submitted': 'avg_time_per_exam',  
                          'score' : 'avg_score_per_exam'})

In [106]:
df_rfm.describe()

,id_student,avg_time_per_exam,completion_rate,avg_score_per_exam
count,4.633000e+03,4633.000000,4633.000000,4633.000000
mean,7.256904e+05,238.462227,0.881286,65.117958
std,5.753498e+05,5.653378,0.321142,20.470561
min,2.369800e+04,229.000000,0.000000,0.000000
25%,5.011580e+05,234.000000,1.000000,50.000000
50%,5.884820e+05,241.000000,1.000000,66.000000
75%,6.463510e+05,243.000000,1.000000,82.000000
max,2.698251e+06,285.000000,1.000000,100.000000


Исходя из распределения данных, выделяю следующие <b>ранги для каждого показателя</b>:<br> 
avg_time_per_exam (R): четыре ранга по 3 перцентилям (до 25%, 25% - 50%, 50% - 75%, выше 75%; чем быстрее сдан экзамен, тем выше ранг)<br>completion_rate (F): три ранга по каждому значению (1 - плохая завершаемость, ни одной удачной попытки, 2 - средняя завершаемость, 3 - хорошая завершаемость, все попытки удачные)<br>avg_score_per_exam (M): четыре ранга, которые примерно соответсвуют переводу стобальной системы в пятибальную шкалу оценки (1 - не сдано до 40 баллов, 2 - "тройка" до 60 баллов, 3 - "четверка" до 85 баллов, 4 - "пятерка" выше 85 быллов)

In [107]:
quintiles = df_rfm['avg_time_per_exam'].quantile([.25, .50, .75]).to_dict()
quintiles

{0.25: 234.0, 0.5: 241.0, 0.75: 243.0}

In [108]:
def r_score(x):
    if x <= quintiles[.25]:
        return 4
    elif x <= quintiles[.50]:
        return 3
    elif x <= quintiles[.75]:
        return 2
    else:
        return 1

In [109]:
def f_score(x):
    if x == 1:
        return 3
    elif x == 0.5:
        return 2
    else:
        return 1

In [110]:
def m_score(x):
    if x >= 85:
        return 4
    elif x >= 60:
        return 3
    elif x >= 40:
        return 2
    else:
        return 1

In [111]:
df_rfm['R'] = df_rfm['avg_time_per_exam'].apply(lambda x: r_score(x))
df_rfm['F'] = df_rfm['completion_rate'].apply(lambda x: f_score(x))
df_rfm['M'] = df_rfm['avg_score_per_exam'].apply(lambda x: m_score(x))

In [112]:
df_rfm['RFM_Score'] = df_rfm['R'].map(str) + df_rfm['F'].map(str) + df_rfm['M'].map(str)

In [113]:
RFM_Score = df_rfm.groupby('RFM_Score', as_index=False).agg({'id_student':'count'})

In [114]:
segt_map = {
    r'[1-2]11': 'самые слабые студенты',
    r'[1-2]21': 'очень слабые результаты',
    r'[1-2]2[2-3]': 'слабые результаты',
    r'[1-2]24': 'нестабильные и долгие результаты (от неуд. до хороших оценок)',
    r'[1-2]3[2-3]': 'средние, но долгие результаты',
    r'[1-2]34': 'отличные, но долгие результаты',
    r'[3-4]11': 'быстрая сдача, но отсутсвие результата',
    r'[3-4]21': 'быстрая сдача, но очень слабые результаты',
    r'[3-4]2[2-3]': 'быстрая сдача, но слабые результаты',
    r'[3-4]24': 'нестабильные быстрые результаты (от неуд. до хороших оценок)',
    r'[3-4]3[2-3]': 'средние быстрые результаты',
    r'[3-4]34': 'самые сильные студенты'
}

df_rfm['Segment'] = df_rfm['R'].map(str) + df_rfm['F'].map(str) + df_rfm['M'].map(str)
df_rfm['Segment'] = df_rfm['Segment'].replace(segt_map, regex=True)

In [115]:
RFM_results = df_rfm.groupby('Segment')\
              .agg({'id_student':'count'})\
              .rename(columns={'id_student':'students_number'})\
              .sort_values('students_number')
RFM_results

,students_number
Segment,
очень слабые результаты,1
слабые результаты,3
"быстрая сдача, но очень слабые результаты",6
"быстрая сдача, но слабые результаты",18
самые слабые студенты,267
"быстрая сдача, но отсутсвие результата",269
самые сильные студенты,427
"отличные, но долгие результаты",484
"средние, но долгие результаты",1452


### Выводы: 
1. Наибольшее количество студентов имеет средние результаты (все попытки сдать экзамен успешные со средней оценков 3-4 балла (в переводе в пятибальную шкалу))
2. Самых сильных студентов (сдали все экзамены на "отлично") гораздно больше, чем самых слабых студентов (не сдали ни одного экзамена успешно)
3. Среднее время сдачи студентом одного экзамена не влияет на успеваемость и не отражается на среднем балле за экзамен